In [1]:
import pandas as pd
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.compat import lzip

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
sales_train_evaluation = pd.read_csv('sales_train_evaluation.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sales_train_evaluation.csv'

In [ ]:
sales_train_validation = pd.read_csv('sales_train_validation.csv')

In [ ]:
sell_prices=pd.read_csv('sell_prices.csv')

In [ ]:
sell_prices.head

In [ ]:
sales_train_validation.head

In [ ]:


sales_train_validation

In [ ]:
df=pd.read_csv('sales_train_validation.csv')

In [ ]:
data=df

In [ ]:
def cal_days(data): #calculate and output a list of days 
    start_day=1
    day_format="d_"
    converted=False
    cols=[]
    for j in range(math.ceil((len(data)-6)//7) ):
        for i in range(1,8):
            cols.append(day_format+str(start_day))
            start_day=start_day+1
    return cols


    

In [ ]:
len(cols)

In [ ]:
def create_week(cols,df): # create columns for each week and quantity of products sold that week
    for i in range(0,len(cols),7):
        temp=cols[i:i+7]

        df["week" +str(int(i/7+1))]=df[temp].sum(axis=1)
    

In [ ]:
create_week(cols,df)
df

In [ ]:
def cal_extra_days(data):# calcualte extra days not included in weeks listed
    if not ((len(data.columns)-6)/7).is_integer():
        extra_days=(len(data.columns)-6)-(((len(data.columns)-6)//7)*7)
    return extra_days

In [ ]:
extra_days=cal_extra_days(data)
extra_days

In [ ]:
def add_extra_days_col(data,cols): # add extra days to list to sum all later to week
    temp_col=[]
    extra_days=cal_extra_days(data) 
    for i in range(((len(cols))//7)*7+1,((len(cols))//7)*7+extra_days+1):
        temp="d_"+str(i)
        temp_col.append(temp)   
    
    data["week" +str((len(cols))//7+1)]=data[temp_col].sum(axis=1)

In [ ]:
add_extra_days_col(df,cols)
df

In [ ]:
df

In [ ]:
def create_week_sales_info(df,cols):
    info_sales=df.iloc[:,:6]
    week_sales=df.iloc[:,-math.ceil((len(cols))/7):]
    result_df = pd.concat([info_sales, week_sales], axis=1)
    return result_df

In [ ]:
result_df=create_week_sales_info(df,cols)
result_df

In [ ]:
result_df

In [ ]:
def create_data_predict(result,sell_data):
    start_week="week"+str((int(str(sell_data.iloc[0,2])[2])*52-52+int(str(sell_data.iloc[0,2])[3:])))
    x=result_df.loc[1][start_week:].values.tolist()
    y=sell_data.iloc[0:146,3].values.tolist()

    return x,y

In [ ]:
x,y=create_data_predict(result_df,sell_prices)

In [ ]:
len(x)

In [ ]:
y

In [ ]:
x=result_df.loc[1][start_week:]

In [ ]:
y=sell_prices.iloc[0:146,3]

In [ ]:
y

In [ ]:
data = pd.concat([x,y], axis=1,ignore_index=False)

In [ ]:
x= sm.add_constant(x)

In [ ]:
model = sm.OLS(y,x)

In [ ]:
results = model.fit()

In [ ]:
results.params

In [ ]:
results.tvalues

In [ ]:
print(results.t_test([1, 0]))

In [ ]:
model=sm.OLS(y, x).fit()

In [ ]:
sm.OLS(y,x).fit().summary()

In [ ]:
plt.rc("figure", figsize=(16, 8))


In [ ]:
plt.rc("font", size=14)

In [ ]:
import seaborn as sns

In [ ]:
p=model.params

In [ ]:
p[0]

In [ ]:
data

In [ ]:
data.rename(columns={1:"quantity"},inplace=True)

In [ ]:
from linearmodels import IV2SLS 

In [ ]:
iv_reg = IV2SLS.from_formula("sell_price ~ quantity", data).fit()


In [ ]:
iv_reg.summary

In [ ]:
test=sell_prices.loc[0:146]
test[]

In [ ]:
price_range=sell_prices.groupby(['store_id','item_id'])['sell_price'].apply(lambda x: max(x) - min(x))
max_change_product=price_range.idxmax()
max_change_value=price_range.max()

In [ ]:
max_change_product

In [ ]:
price_range.max()

In [ ]:
max_change_value

In [ ]:
test=sell_prices.loc[sell_prices["store_id"]=="WI_3" ]

In [ ]:
test=test.loc[sell_prices["item_id"]=="HOUSEHOLD_2_406"]

In [ ]:
test

In [ ]:
display(test.to_string())

In [ ]:
dates=test.loc[test["sell_price"] >13]["wm_yr_wk"]

In [ ]:
start_week="week"+str((int(str(sell_prices.iloc[0,2])[2])*52-52+int(str(sell_prices.iloc[0,2])[3:])))
x=result_df.loc[1][start_week:].values.tolist()
y=sell_prices.iloc[0:146,3].values.tolist()

In [ ]:
start_week=dates.iloc[0]


In [ ]:
start_week
x=result_df.loc[1][start_week:].values.tolist()


In [ ]:
result=result_df.loc[result_df["item_id"]=="HOUSEHOLD_2_406"]

In [ ]:
result=result.loc[result["store_id"]=="WI_3"]

In [ ]:
x=result.iloc[:,6:].values.tolist()

In [ ]:
x=x[0]

In [ ]:
x[115:125]

In [ ]:
y=sell_prices.iloc[0:146,3].values.tolist()

In [ ]:
test=test["sell_price"].values.tolist()

In [ ]:
test[121]

In [ ]:
x= sm.add_constant(x)

In [ ]:
sm.OLS(test[0:-8],x).fit().summary()

In [ ]:
#Demand Forecasting Attempt